# Import

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np 

from sklearn.impute import KNNImputer


from string import ascii_lowercase
from itertools import combinations
from sklearn.preprocessing import OneHotEncoder

from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import  GradientBoostingClassifier
import lightgbm as lgb

from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Data Load

In [2]:
#데이터 불러오기
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

# Q

In [3]:
#전체 Answers와 secret 제외한 Q_Answers 정의
Answers = []
for i in range(1, 27):
    Answers.append('Q' + str(i))
    
Q_Answers = []
for i in range(1, 21):
    Q_Answers.append('Q' + str(i))

In [4]:
#결측치 예측에 KNN 사용
def knull(col):
    imputer = KNNImputer(n_neighbors=3)
    a = imputer.fit_transform(train[col])
    train[col] = a

In [5]:
knull(Answers)

In [6]:
for i in range(len(Answers)):
    test[Answers[i]]=test[Answers[i]].fillna(train[Answers[i]].mode()[0])

In [7]:
# 1~20번 항목 더해서 Q_score, 21~26번 항목 더해서 Qs_score 생성
Q_secret= ['Q21', 'Q22', 'Q23', 'Q24', 'Q25', 'Q26']

dataset = [train, test]

for data in dataset:
    data['Q_score'] = data[Q_Answers].sum(axis=1)
    data['Qs_score'] = data[Q_secret].sum(axis=1)

In [8]:
# 마키아벨리즘 테스트(1~20번 항목)에서 T, V, M 컬럼 생성
for data in dataset:
    data['T'] = data['Q1']+data['Q2']+data['Q3']+data['Q7']+data['Q10']+data['Q12']+data['Q15']+data['Q16'] 
    data['V'] = data['Q4']+data['Q5']+data['Q8']+data['Q11']+data['Q13']+data['Q20']
    data['M'] = data['Q9']+data['Q19']

In [9]:
# 마키아벨리즘 스코어, 분산 피처 생성
for data in dataset:
    data['Mach_score'] = data[Answers].mean(axis = 1)
    data['Mach_var'] = data[Answers].var(axis = 1)

In [10]:
# 마키아벨리즘 비율 피처 생성
Ancoms = list(combinations(Answers, 2))
for data in dataset:
    for a,b in Ancoms:
        data['mach_%s_dv%s'%(a,b)] = data[a]/data[b]

C:\Users\thdal\AppData\Local\Temp/ipykernel_4252/4282676114.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data['mach_%s_dv%s'%(a,b)] = data[a]/data[b]


In [11]:
# Q drop
for data in dataset:
    data.drop([('Q'+str(i) )for i in range(1,27)], axis = 1, inplace = True)

# TIPI

In [12]:
TIPI = []
for i in range(1,11):
    TIPI.append('TIPI'+str(i))

In [13]:
knull(TIPI)

In [14]:
for i in range(len(TIPI)):
    test[TIPI[i]]=test[TIPI[i]].fillna(train[TIPI[i]].mode()[0])

In [15]:
# tp1~10 항목 사용해 Ex, Ag, Con, Es, Op 컬럼 만들기

train['Extraverted'] = train['TIPI1']-train['TIPI6']
train['Warm'] = train['TIPI7']-train['TIPI2']
train['Dependable'] = train['TIPI3']-train['TIPI8']
train['Calm'] = train['TIPI9']-train['TIPI4']
train['OpenMind'] = train['TIPI5']-train['TIPI10']

test['Extraverted'] = test['TIPI1']-test['TIPI6']
test['Warm'] = test['TIPI7']-test['TIPI2']
test['Dependable'] = test['TIPI3']-test['TIPI8']
test['Calm'] = test['TIPI9']-test['TIPI4']
test['OpenMind'] = test['TIPI5']-test['TIPI10']

In [16]:
# tp 비율 피처 생성
tpcoms = list(combinations(TIPI, 2))

for data in dataset:
    for a,b in tpcoms:
        data['%s_dv_%s'%(a,b)] = data[a]/data[b]

In [17]:
# 기존 TIPI drop
train.drop([('TIPI'+str(i)) for i in range(1,10)], axis=1, inplace = True)
train.drop('TIPI10', axis = 1, inplace = True)

test.drop([('TIPI'+str(i)) for i in range(1,10)], axis=1, inplace = True)
test.drop('TIPI10', axis = 1, inplace = True)

# Index

In [18]:
#index drop
for data in dataset:
    data.drop('index', axis=1, inplace=True)

# Country

In [19]:
#country 5개 구간으로 나누기
def country_label(data):
    nara = data['country'].copy()
    nara_val = nara.value_counts()
    
    a = [] 
    b = []
    c = [] 
    d = []
    e = []

    
    for i in range(len(nara_val)):
        if nara_val.values[i] <= 10 : a.append(nara_val.index[i])
        elif 100>nara_val.values[i] >10 : b.append(nara_val.index[i])
        elif 500>nara_val.values[i] >= 100 :c.append(nara_val.index[i])
        elif 5000>nara_val.values[i] >= 500 :  d.append(nara_val.index[i])
        elif nara_val.values[i] >= 5000 : e.append(nara_val.index[i])

    for i in range(len(nara)):
        if nara[i] in a: nara[i]=0
        if nara[i] in b: nara[i]=1
        if nara[i] in c: nara[i]=2
        if nara[i] in d: nara[i]=3
        if nara[i] in e: nara[i]=4

        
    nara.fillna(0, inplace = True)
    return nara

In [20]:
train['nara'] = country_label(train)
test['nara'] = country_label(test)

In [21]:
train = train.drop(columns=['country'], axis=1)
test = test.drop(columns=['country'], axis=1)

# VCL

In [22]:
VCL = []
for i in range(1,17):
    VCL.append('VCL'+str(i))

In [23]:
#작성자의 영어 실력

# 실제로 존재하고 쉬운단어
VCL_1 = ['VCL1','VCL4','VCL5','VCL10','VCL14','VCL16']
# 실제로 있는데 어려운 단어
VCL_2 = ['VCL2','VCL3','VCL7','VCL8','VCL11','VCL13','VCL14']
# 세상에 없는 단어
VCL_3 = ['VCL6','VCL9','VCL12']

In [24]:
#VCL score 생성

dataset = [train,test]
for data in dataset:
    data['VCL_score'] = data[VCL].sum(axis=1)
    data['VCL_3_score'] = data[VCL_3].sum(axis=1)
    data.drop([('VCL'+str(i) )for i in range(1,17)], axis = 1, inplace = True)

# Hand

In [25]:
train.drop('hand', axis=1, inplace = True)
test.drop('hand', axis=1, inplace = True)

In [26]:
#이상치 제거
train = train.drop(train[train.familysize > 50].index)

# 결측치 대체

In [27]:
#train 데이터 결측치 대체
knull(['familysize','religion','orientation','married','voted','nerdiness'])

# Age

In [28]:
#train 데이터의 age가 100이상인 값 이상치로 간주하고 제거
train = train.drop(train[train.age > 100].index)

In [29]:
# age_cuts 연령대별 피처 생성
def age_categorize(age):
    age = (age // 10)
    return age

train['age_cuts'] = age_categorize(train['age'])
test['age_cuts'] = age_categorize(test['age'])

# 기존의 age drop하기
train.drop('age', axis=1, inplace=True)
test.drop('age', axis=1, inplace = True)

# Elapse

In [30]:
elapse = ['introelapse', 'testelapse', 'surveyelapse']

In [31]:
train[elapse]=np.log1p(train[elapse])

In [32]:
#걸린시간 합
train['elapse_sum'] = train['testelapse']+train['surveyelapse']
test['elapse_sum'] = test['testelapse']+test['surveyelapse']

train.drop('testelapse', axis=1, inplace = True)
test.drop('testelapse', axis=1, inplace = True)

train.drop('surveyelapse', axis=1, inplace = True)
test.drop('surveyelapse', axis=1, inplace = True)

# ASD

In [33]:
#ASD 결측치 최빈값 대체
train['ASD'] = train['ASD'].fillna(train['ASD'].mode()[0])
test['ASD'] = test['ASD'].fillna(train['ASD'].mode()[0])

#education 결측치 최빈값 대체
train['education'] = train['education'].fillna(train['education'].mode()[0])
test['education'] = test['education'].fillna(train['education'].mode()[0])

#engnat 결측치 최빈값 대체
train['engnat'] = train['engnat'].fillna(train['engnat'].mode()[0])
test['engnat'] = test['engnat'].fillna(train['engnat'].mode()[0])

#gender 결측치 최빈값 대체
train['gender'] = train['gender'].fillna(train['gender'].mode()[0])
test['gender'] = test['gender'].fillna(train['gender'].mode()[0])

#religion 결측치 최빈값 대체
test['religion'] = test['religion'].fillna(train['religion'].mode()[0])

#orientation 결측치 최빈값 대체
test['orientation'] = test['orientation'].fillna(train['orientation'].mode()[0])

#voted 결측치 최빈값 대체
test['voted'] = test['voted'].fillna(train['voted'].mode()[0])

#married 결측치 최빈값 대체
test['married'] = test['married'].fillna(train['married'].mode()[0])

#familysize 결측치 최빈값 대체
test['familysize'] = test['familysize'].fillna(train['familysize'].mode()[0])

# Label Encoding

In [34]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
needenco = ['gender', 'religion', 'orientation']
for i in needenco:
    train[i] = encoder.fit_transform(train[i])
    test[i] = encoder.transform(test[i])

# Model 1: LGBM Ensemble

In [39]:
# 타겟 분리
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [40]:
# LGBM 1
def lgbm_rfe_30(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=30)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [41]:
import warnings

warnings.filterwarnings(action='ignore')
#warnings.filterwarnings(action='default')

In [42]:
lgbm_30 = lgbm_rfe_30(train_x, train_y)

398 0.8524157335074692
358 0.8446875860170657
322 0.8497890098550365
289 0.8442191349173718
260 0.8493205587553425
234 0.8517874850135957
210 0.8437645041328501
189 0.8478250854844077
170 0.8528145071848363
153 0.8452689030240002
137 0.8479071436057433
123 0.8455245788546876
110 0.8444100280206891
99 0.8449182125265033
89 0.8465314463996351
80 0.8458476287218396
72 0.848468018063152
64 0.8424835336703187
57 0.8375148424426486
51 0.8393451704332784
45 0.8396751304580168
40 0.8381551837122979


In [43]:
model1 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_1 = train_x[lgbm_30.iloc[7,2]]
model1.fit(x_train_1, train_y)

pred_y1 = model1.predict_proba(test[lgbm_30.iloc[7,2]])
pred_y1 = pred_y1[:,1]

In [44]:
# LGBM 2
def lgbm_rfe_500(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=500)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [45]:
lgbm_500 = lgbm_rfe_500(train_x, train_y)

398 0.8477122281224995
358 0.848088356362889
322 0.8469123422022236
289 0.83966083220466
260 0.8471833970124507
234 0.8481801512826463
210 0.8438017068081811
189 0.8509111944770633
170 0.8426554249642924
153 0.84178972381853
137 0.8467824437308689
123 0.8408437742837975
110 0.8434206712922069
99 0.8457363280415583
89 0.841245882218458
80 0.841099241277462
72 0.8376315294188286
64 0.8266744490284171
57 0.8423003691425893
51 0.8369505727656595
45 0.8363686276139901
40 0.8324780240652794


In [46]:
model2 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_2 = train_x[lgbm_500.iloc[14,2]]
model2.fit(x_train_2, train_y)

pred_y2 = model2.predict_proba(test[lgbm_500.iloc[14,2]])
pred_y2 = pred_y2[:,1]

In [47]:
# LGBM 3
def lgbm_rfe_913(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=913)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [48]:
lgbm_913 = lgbm_rfe_913(train_x, train_y)

398 0.8535521747313413
358 0.8549374697625225
322 0.8497525001125786
289 0.8563455691920085
260 0.8542172549046776
234 0.8566506862680574
210 0.8528712180781722
189 0.8527730725664819
170 0.8501725051699592
153 0.8524711307863997
137 0.8544649279900607
123 0.8481925638032424
110 0.8540310670957357
99 0.8546488064928451
89 0.849245030084486
80 0.8514371389544155
72 0.85048483882775
64 0.8436588184896906
57 0.8432275555647928
51 0.8454802837213545
45 0.846106971679824
40 0.8474033584239449


In [49]:
model3 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_3 = train_x[lgbm_913.iloc[7,2]]
model3.fit(x_train_3, train_y)

pred_y3 = model3.predict_proba(test[lgbm_913.iloc[7,2]])
pred_y3 = pred_y3[:,1]

In [50]:
# LGBM 4
def lgbm_rfe_8(data_x, data_y, ratio=0.9, min_feats=40):
    feats = data_x.columns.tolist()
    archive = pd.DataFrame(columns=['model', 'n_feats', 'feats', 'score'])
    while True:
        model = LGBMClassifier(objective = 'binary', num_iterations=10**4)
        x_train, x_val, y_train, y_val = train_test_split(data_x[feats], data_y, random_state=8)
        model.fit(x_train, y_train, eval_set = [(x_val, y_val)], early_stopping_rounds=100, verbose=False)
        val_pred = model.predict_proba(x_val)
        val_pred = val_pred[:,1]
        score = roc_auc_score(y_val, val_pred)
        n_feats = len(feats)
        print(n_feats, score)
        archive = archive.append({'model':model, 'n_feats':n_feats, 'feats':feats, 'score':score}, ignore_index=True)
        feat_imp = pd.Series(model.feature_importances_, index=feats).sort_values(ascending=False)
        next_n_feats = int(n_feats*ratio)
        if next_n_feats < min_feats:
            break
        else:
            feats = feat_imp.iloc[:next_n_feats].index.tolist()
    return archive

In [51]:
lgbm_8 = lgbm_rfe_8(train_x, train_y)

398 0.8531349325320109
358 0.8528316798018748
322 0.8473145477456492
289 0.848286909245877
260 0.8457150044626397
234 0.8472054227101834
210 0.8449967894265881
189 0.8501374401104577
170 0.8438541354368044
153 0.8474874247755183
137 0.8474871376043722
123 0.8494511010716079
110 0.847056668056575
99 0.8512752121907597
89 0.8411963664809244
80 0.8456320120014565
72 0.8461517917756481
64 0.8341308076056704
57 0.8361048220630145
51 0.8383464800283954
45 0.8402165385309013
40 0.8295768475730019


In [52]:
model4 = LGBMClassifier(objective="binary", num_iterations=10**3)
x_train_4 = train_x[lgbm_8.iloc[8,2]]
model4.fit(x_train_4, train_y)

pred_y4 = model4.predict_proba(test[lgbm_8.iloc[8,2]])
pred_y4 = pred_y4[:,1]

### LGBM Ensemble

In [53]:
pred_all = (pred_y1 + pred_y2 + pred_y3 + pred_y4) * (1/4)

# Model 2: ExtraTrees Regressor

In [54]:
# 타겟 분리
train_x = train.copy()
train_x.drop('nerdiness', axis=1, inplace = True)
train_y = train['nerdiness']

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

In [55]:
# 하이퍼파라미터 튜닝
n_estimators = int(391.707369561944)
max_depth = int(65.33353663762796)
min_samples_split = int(5.797880430957836)
min_samples_leaf = int(3.813420188476544)
bootstrap = bootstrap = 0
    
assert type(n_estimators) == int
assert type(max_depth) == int
assert type(min_samples_split) == int
assert type(min_samples_leaf) == int

In [56]:
#Extra Trees Regressor
from sklearn.ensemble import ExtraTreesRegressor

xtree_tune = ExtraTreesRegressor(n_estimators=n_estimators,
                               max_depth=max_depth,
                               min_samples_split=min_samples_split,
                               min_samples_leaf=min_samples_leaf,
                               #max_features=max_features,
                               bootstrap=bootstrap,
                               oob_score=bootstrap,
                               n_jobs=6,
                               random_state=42,
                               verbose=0)
xtree_tune.fit(train_x, train_y)

preds_tune = xtree_tune.predict(test)

# Ensemble: Model1 + Model2

In [57]:
pred_y_ensemble = (preds_tune)*(0.75) + (pred_all)*(0.25)

# 제출 파일 생성

In [58]:
submission = pd.DataFrame({
    "nerdiness" : pred_y_ensemble
})
submission.to_csv('sub_0826.csv', index=True)

In [59]:
subb = pd.read_csv('sub_0826.csv')

In [60]:
subb.rename(columns = {'Unnamed: 0' : 'index'}, inplace = True)

In [61]:
subb.set_index('index', inplace = True)

In [62]:
subb

,nerdiness
index,
0,0.140858
1,0.783248
2,0.899717
3,0.718801
4,0.912121
...,...
35447,0.938553
35448,0.728430
35449,0.965168


In [64]:
subb.to_csv("sub_0826_lgbm_ext.csv")